In [213]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [207]:
train_df = pd.read_csv("train.csv")

In [208]:
private_test_df = pd.read_csv("test.csv")

In [87]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train_df, test_size=0.2)

In [209]:
"""
X_train = train_df["title"].values
y_train = train_df["target"].astype(int).values
X_private_test = private_test_df["title"].values
"""

In [88]:
X_train = train_df["title"].values
y_train = train_df["target"].astype(int).values

X_test = test_df["title"].values
y_test = test_df["target"].astype(int).values

X_private_test = private_test_df["title"].values

Сначала пробовал метод опорных векторов, потом лог регрессию

In [214]:
from sklearn.linear_model import SGDClassifier

In [215]:
text_clf = Pipeline([
     ('vect', CountVectorizer(ngram_range=(1,2))),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-6, random_state=42,
                           max_iter=5, tol=None, learning_rate='optimal')),
])

In [216]:
from sklearn.linear_model import LogisticRegression 

In [222]:
text_clf = Pipeline([
     ('vect', CountVectorizer(ngram_range=(1,2))),
     ('tfidf', TfidfTransformer()),
     ('clf', LogisticRegression(C = 500, solver='saga',n_jobs=-1)),
])

In [228]:
%%time

text_clf.fit(X_train, y_train)  
y_train_pred = text_clf.predict(X_train)

/home/administrator/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


CPU times: user 1min, sys: 276 ms, total: 1min
Wall time: 28.7 s


In [230]:
print("f1 = ", f1_score(y_train_pred, y_train))

f1 =  0.9953223388305849


In [231]:
y_test_pred = text_clf.predict(X_test)


In [232]:
print("f1 = ", f1_score(y_test, y_test_pred))

f1 =  0.9939320388349515


# Submit

In [212]:
private_test_df["target"] = text_clf.predict(X_private_test).astype(bool)

private_test_df[["id", "target"]].to_csv("submission.csv", index=False)

!cat submission.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: ошибка записи: Обрыв канала
